In [1]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [48]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
#from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset

import cassio

In [3]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [4]:
from PyPDF2 import PdfReader

In [5]:
ASTRA_DB_APPLICATION_TOKEN="AstraCS:tIFJtOqbDBPURdfPbISFXxar:cd1bb55a3483450b101dae40736cca2d7c8413b87138e2fc4606054900396"
ASTRA_DB_ID="78459230-d954-46e9-9f67362e61"
OPENAI_API_KEY="OPENAI_API_KEY"

In [7]:
pdfreader=PdfReader('Envision.pdf')

In [8]:
from typing_extensions import Concatenate
raw_text=''
for i,page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text+=content

In [9]:
print(raw_text)

 
 
 
CM Guide  
 
Introduction:  
• Aaruush means the “First Rays of Sun”  
• Our Theme - Towards Infinity  
• Our Motto - Rising in the spirit of innovation  
• Our logo, that is of a rising sun, implies that we will shed light on the most 
unexpected realms of Science and Technology.  
 
Patrons of Aaruush:  
● Dr. C. Muthamizhchelvan (Vice -Chancellor, SRMIST)  
● Prof. T.V. Gopal (Dean - College of Engg. & Tech.)  
● Dr. A. Rathinam (Convenor of Aaruush, Director of Alumni Affairs)  
● Dr. V. Thirumurugan (Associate Director - Campus Life)  
 
Brief History of Aaruush : 
• Aaruush started off as a technical fest in 2007.  
• It was inaugurated in 2007 by Late Ex -President Dr. APJ Abdul Kalam.  
• In 2009, Aaruush evolved from a technical fest to the National Techno -Management 
Fest.  
 
Date of Aaruush’22 Main Days:  
27th October - 30th October, 20 22 
 
Structure of Aaruush:  
Aaruush comprises of : 
• 14 committees   
 
• 14 domains  
• 1 Team  
 
The Committees, Domains, Tea

connection to database

In [10]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(135002215821904) 78459230-d954-46e9-9f6a-07d9a7362e61-us-east1.db.astra.datastax.com:29042:6e2342c8-6769-4177-b9e6-07bc4ca8148f> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


LangChain Embedding or LLM object

In [57]:
llm= OpenAI(openai_api_key='OPENAI_API_KEY')


In [58]:
embedding = OpenAIEmbeddings(openai_api_key='OPENAI_API_KEY')

In [51]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name='qa_mini_demo',
    session=None,
    keyspace=None,
)

In [53]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)

texts = text_splitter.split_text(raw_text)

In [54]:
texts[:50]

['CM Guide  \n \nIntroduction:  \n• Aaruush means the “First Rays of Sun”  \n• Our Theme - Towards Infinity  \n• Our Motto - Rising in the spirit of innovation  \n• Our logo, that is of a rising sun, implies that we will shed light on the most \nunexpected realms of Science and Technology.  \n \nPatrons of Aaruush:  \n● Dr. C. Muthamizhchelvan (Vice -Chancellor, SRMIST)  \n● Prof. T.V. Gopal (Dean - College of Engg. & Tech.)  \n● Dr. A. Rathinam (Convenor of Aaruush, Director of Alumni Affairs)  \n● Dr. V. Thirumurugan (Associate Director - Campus Life)  \n \nBrief History of Aaruush : \n• Aaruush started off as a technical fest in 2007.  \n• It was inaugurated in 2007 by Late Ex -President Dr. APJ Abdul Kalam.  \n• In 2009, Aaruush evolved from a technical fest to the National Techno -Management',
 '• It was inaugurated in 2007 by Late Ex -President Dr. APJ Abdul Kalam.  \n• In 2009, Aaruush evolved from a technical fest to the National Techno -Management \nFest.  \n \nDate of Aaruush

In [55]:
astra_vector_store.add_texts(texts[:50])
print("Inserted %i headlines." %len(texts[:50]))
astra_vector_index =VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 35 headlines.


Q/A session start

In [59]:
f_q=True
while True:
  if f_q:
    query_text=input("\nEnter your Question [or type quit to exit]:").strip()
  else:
    query_text =input("\n What is your Next Question:").strip()

  if query_text.lower()=="quit":
    break

  if query_text=="":
    continue

  f_q=False

  print("\nQuestion:\"%s\""%query_text)
  answer=astra_vector_index.query(query_text, llm=llm).strip()
  print("answer:\"%s\"\n"%answer)

  print("first doc by relevence:")
  for doc,score in astra_vector_store.similarity_search_with_score(query_text, k=4):
    print(" [%0.4f]\"%s...\""%(score,doc.page_content[:84]))



Enter your Question [or type quit to exit]:what is team envision?

Question:"what is team envision?"


answer:"Team Envision is a Multidisciplinary Technical Team of Aaruush that aims to provide solutions to campus and societal issues through the development of products and research papers. They also provide technical support for Aaruush events and have worked on projects related to campus and social causes, such as the A'19 website and Aaruush application."

first doc by relevence:


 [0.9415]"Team Envision  
Team Envision is a Multidisciplinary Te chnical Team of Aaruush that..."
 [0.9415]"Team Envision  
Team Envision is a Multidisciplinary Te chnical Team of Aaruush that..."
 [0.9145]"develop products to tackle campus and societa l issues and problems. These products ..."
 [0.9145]"develop products to tackle campus and societa l issues and problems. These products ..."

 What is your Next Question:events organised by envision



Question:"events organised by envision"


answer:"Team Envision has provided technical support for all Aaruush Events, such as the A'19 Website and the Aaruush Application (Android & iOS). They have also participated in various hackathons and presented solutions to campus and societal issues."

first doc by relevence:
 [0.9218]"Team Envision  
Team Envision is a Multidisciplinary Te chnical Team of Aaruush that..."
 [0.9218]"Team Envision  
Team Envision is a Multidisciplinary Te chnical Team of Aaruush that..."
 [0.9126]"develop products to tackle campus and societa l issues and problems. These products ..."
 [0.9125]"develop products to tackle campus and societa l issues and problems. These products ..."

 What is your Next Question:Teachbot

Question:"Teachbot"


answer:"Teachbot is a division that focuses on developing technologies that mimic human teaching abilities and serve social causes. This includes creating automatic smart systems, working on OpenCV projects, and using machine learning technologies to solve daily life problems and issues on college campuses. They also work on developing mobile applications, such as the Near You App and Here For You, which addresses mental health tracking. Blockchain is another division that focuses on the development of blockchain technology."

first doc by relevence:
 [0.8965]"Organiser: Harsh Bardhan  
Email:  praesentatio@aaruush.net  
 
11) Robogyan:  
Robo..."
 [0.8965]"Organiser: Harsh Bardhan  
Email:  praesentatio@aaruush.net  
 
11) Robogyan:  
Robo..."
 [0.8932]"technologies that work exactly like humans and serve social causes. Be it developing..."
 [0.8932]"technologies that work exactly like humans and serve social causes. Be it developing..."

 What is your Next Question:projects done by t


Question:"projects done by team envision"


answer:"Some of the projects done by Team Envision include developing websites for A'19 and A'20, creating the Aaruush application for Android and iOS, designing the Suroma House of Children website, developing the Near You app, and working on various projects related to AI/ML, IoT, and solar energy."

first doc by relevence:
 [0.9422]"Team Envision  
Team Envision is a Multidisciplinary Te chnical Team of Aaruush that..."
 [0.9422]"Team Envision  
Team Envision is a Multidisciplinary Te chnical Team of Aaruush that..."
 [0.9369]"● Projects related to the campus and social causes.  
 
Projects Done By Team Envisi..."
 [0.9369]"● Projects related to the campus and social causes.  
 
Projects Done By Team Envisi..."

 What is your Next Question:quit
